In [1]:
!pip install PyGithub
import pandas as pd
import unicodedata
from datetime import datetime
import requests
from io import StringIO
from github import Github
from getpass import getpass
from datetime import datetime
import functools as ft

##urls

In [2]:
url_ordr_itms = 'https://raw.githubusercontent.com/Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251/main/projeto/development/silver/order_items_tratados.csv'
url_prdt = 'https://raw.githubusercontent.com/Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251/main/projeto/development/silver/order_products_tratados.csv'
url_ordr_rvw = 'https://raw.githubusercontent.com/Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251/main/projeto/development/silver/order_reviews_tratados.csv'
url_ordr_sell = 'https://raw.githubusercontent.com/Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251/main/projeto/development/silver/order_sellers_tratados.csv'
url_ordr = 'https://raw.githubusercontent.com/Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251/main/projeto/development/silver/orders_tratados.csv'

##dataframes

In [3]:
df_ordr_itms = pd.read_csv(url_ordr_itms,index_col='order_id')
df_prdt = pd.read_csv(url_prdt,index_col='product_id')
df_ordr_rvw = pd.read_csv(url_ordr_rvw,index_col='order_id')
df_ordr_sell = pd.read_csv(url_ordr_sell,index_col='seller_id')
df_ordr = pd.read_csv(url_ordr,index_col='order_id')

## FACT TEMPO

In [4]:
fct_tempo = pd.merge(
    pd.merge(df_ordr, df_ordr_itms, on='order_id', how='left'),
    df_prdt, on='product_id', how='left'
)
# TRATAMENTO NAN
fct_tempo.fillna({
    'order_estimated_delivery_date': '99/99/9999',
    'order_delivered_customer_date': '99/99/9999',
    'order_delivered_carrier_date': '99/99/9999',
    'order_approved_at': '99/99/9999',
    'order_purchase_timestamp': '99/99/9999',
    'product_id': '-',
}, inplace=True)

# ESCOLHER COLUNAS
fct_tempo = fct_tempo[['order_estimated_delivery_date', 'order_delivered_customer_date', 'order_delivered_carrier_date', 'order_approved_at', 'order_purchase_timestamp', 'product_category_name']]

# CONVERTER COLUNAS PARA DATE TIME
fct_tempo['order_estimated_delivery_date'] = pd.to_datetime(fct_tempo['order_estimated_delivery_date'], format='%d/%m/%Y')
fct_tempo['order_delivered_customer_date'] = pd.to_datetime(fct_tempo['order_delivered_customer_date'], format='%d/%m/%Y')
fct_tempo['order_delivered_carrier_date'] = pd.to_datetime(fct_tempo['order_delivered_carrier_date'], format='%d/%m/%Y')
fct_tempo['order_approved_at'] = pd.to_datetime(fct_tempo['order_approved_at'], format='%d/%m/%Y')
fct_tempo['order_purchase_timestamp'] = pd.to_datetime(fct_tempo['order_purchase_timestamp'], format='%d/%m/%Y')

# MEDIA DE TEMPO DE ENTREGA EM DIAS
fct_tempo['avg_delv_days'] = fct_tempo['order_estimated_delivery_date'] - fct_tempo['order_purchase_timestamp']

In [5]:
fct_tempo.head(5)

,order_estimated_delivery_date,order_delivered_customer_date,order_delivered_carrier_date,order_approved_at,order_purchase_timestamp,product_category_name,avg_delv_days
0,2017-10-18,2017-10-10,2017-10-04,2017-10-02,2017-10-02,UTILIDADESDOMESTICAS,16 days
1,2018-08-13,2018-08-07,2018-07-26,2018-07-26,2018-07-24,PERFUMARIA,20 days
2,2018-09-04,2018-08-17,2018-08-08,2018-08-08,2018-08-08,AUTOMOTIVO,27 days
3,2017-12-15,2017-12-02,2017-11-22,2017-11-18,2017-11-18,PETSHOP,27 days
4,2018-02-26,2018-02-16,2018-02-14,2018-02-13,2018-02-13,PAPELARIA,13 days


## FACT REVIEW

In [15]:
final_df = pd.merge(
    pd.merge(
        pd.merge(df_ordr, df_ordr_itms, on='order_id', how='left'),
        df_ordr_rvw, on='order_id', how='left'
    ),
    df_prdt, on='product_id', how='left'
)


In [16]:
# fct_rvw_columns = df_ordr_rvw.join(df_ordr)

# TRATAMENTO NAN
final_df.fillna({
    'order_estimated_delivery_date': '99/99/9999',
    'order_delivered_customer_date': '99/99/9999',
    'order_delivered_carrier_date': '99/99/9999',
    'order_approved_at': '99/99/9999',
    'order_purchase_timestamp': '99/99/9999',
    'review_id': '-',
    'review_score': '-',
    'review_comment_title': '-',
    'review_comment_message': '-',
    'customer_id': '-',
    'order_status': '-',
    'product_category_name': '-',
    'product_id': '-',
    'seller_id': '-',
    'order_item_id': '-',
    'price': '0.00',
    'freight_value': '0.00'
}, inplace=True)

# ESCOLHER COLUNAS
cols = ['customer_id', 'order_id','review_id', 'review_score', 'review_comment_message', 'review_comment_title','product_id', 'product_category_name',
        'order_status', 'seller_id', 'order_item_id', 'price', 'freight_value']
        # 'order_estimated_delivery_date', 'order_delivered_customer_date', 'order_delivered_carrier_date', 'order_approved_at', 'order_purchase_timestamp']

cols_existentes = [col for col in cols if col in final_df.columns]
fct_rvw_columns = final_df[cols_existentes]


In [18]:
fct_rvw_columns.head(5)

,customer_id,review_id,review_score,review_comment_message,review_comment_title,product_id,product_category_name,order_status,seller_id,order_item_id,price,freight_value
0,9EF432EB6251297304E76186B10A928D,-,-,-,-,87285B34884572647811A353C7AC498A,UTILIDADESDOMESTICAS,DELIVERED,3504C0CB71D7FA48D967E0E4C94D59D9,1,29.99,8.72
1,B0830FB4747A6C6D20DEA0B8C802D7EF,8D5266042046A06655C8DB133D120BA5,4.0,MUITO BOM O PRODUTO,MUITO BOA A LOJA,595FAC2A385AC33A80BD5114AEC74EB8,PERFUMARIA,DELIVERED,289CDB325FB7E7F891C38608BF9E0962,1,118.70,22.76
2,41CE2A54C0B03BF3443C3D931A367089,-,-,-,-,AA4383B373C6ACA5D8797843E5594415,AUTOMOTIVO,DELIVERED,4869F7A5DFA277A7DCA6462DCF3B52B2,1,159.90,19.22
3,F88197465EA7920ADCDBEC7375364D82,-,-,-,-,D0B61BFB1DE832B15BA9D266CA96E5B0,PETSHOP,DELIVERED,66922902710D126A0E7D26B0E3805106,1,45.00,27.20
4,8AB97904E6DAEA8866DBDBC4FB7AAD2C,-,-,-,-,65266B2DA20D04DBE00C5C2D3BB7859E,PAPELARIA,DELIVERED,2C9E548BE18521D1C43CDE1C582C6DE8,1,19.90,8.72
